In [1]:
import torch

In [10]:
x = torch.randn(2,requires_grad=True).type(torch.float32)
g = torch.tensor([0,1],requires_grad=False).type(torch.float32)
l = torch.sum(x*g)
l.backward()

In [11]:
print(x.grad)
x.grad = None

tensor([0., 1.])
